In [1]:
from pathlib import Path
import sys, importlib, warnings
warnings.filterwarnings("ignore")

# locate repo root
cwd = Path().resolve()
root = None
for p in [cwd, *cwd.parents]:
    if (p/"src").is_dir():
        root = p; break
assert root
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

import pandas as pd
import numpy as np

import src.config, src.backtest.backtester, src.utils.plotting
importlib.reload(src.config); importlib.reload(src.backtest.backtester); importlib.reload(src.utils.plotting)

from src.config import Settings
from src.backtest.backtester import Backtester, BacktestConfig
from src.utils.plotting import Plotter

cfg = Settings()
cfg




Settings(start='2015-07-01', end='2025-07-31', tickers=['TSLA', 'BND', 'SPY'], risk_free_rate=0.02, seed=42, data_raw_dir=WindowsPath('../data/raw'), data_processed_dir=WindowsPath('../data/processed'), reports_figures_dir=WindowsPath('../reports/figures'))

In [2]:
# Load processed features (Task 1)
features = pd.read_csv(cfg.data_processed_dir / "merged_features.csv", parse_dates=["Date"]).set_index("Date")
rets = features[["TSLA_ret","BND_ret","SPY_ret"]].rename(columns={"TSLA_ret":"TSLA","BND_ret":"BND","SPY_ret":"SPY"}).dropna()

# Choose strategy weights from Task 4 (pick Max Sharpe by default)
weights_path = cfg.reports_figures_dir.parent / "interim" / "weights_max_sharpe.csv"
if not weights_path.exists():
    # fallback to Min Vol if Max Sharpe not generated
    weights_path = cfg.reports_figures_dir.parent / "interim" / "weights_min_vol.csv"

w = pd.read_csv(weights_path, header=None, index_col=0).squeeze()
strategy_weights = {k: float(v) for k, v in w.items() if k in ["TSLA","BND","SPY"]}

strategy_weights


{'TSLA': 0.0, 'BND': 0.0, 'SPY': 1.0}

In [ ]:
cfg_bt = BacktestConfig(
    start="2024-08-01",
    end="2025-07-31",
    rebalance="none",      # or "monthly"
    rf_annual=cfg.risk_free_rate
)

bt = Backtester(rets, cfg_bt)
res = bt.run(strategy_weights=strategy_weights, benchmark_weights={"SPY":0.60,"BND":0.40})

display(res.stats)
res.cumrets.tail()
